In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /DX.csv')


In [ ]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [ ]:
train.shape

In [ ]:
val.shape

In [ ]:
test.shape

In [ ]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 1, 0)
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 



In [ ]:
train = clean(train)
val = clean(val)
test = clean(test)


In [ ]:
train['Raised Series A'].value_counts()

In [ ]:
train.shape , val.shape, test.shape

In [ ]:
train['Investor Names'].isnull().sum()

In [ ]:
def split_columns(text):
    text = text.copy()
    # new data frame with split value columns 
    new = text["Organization Industries"].str.split(",", n = 7, expand = True) 
    # making separate first name column from new data frame 
    text["Primary Industry"]= new[0] 
    # making separate last name column from new data frame 
    text["Sub_Ind"]= new[1] 
    text["Sub_Ind2"]= new[2]
    text["Sub_Ind3"]= new[3]
    text["Sub_Ind4"]= new[4]
    text["Sub_Ind5"]= new[5]
    text["Sub_Ind6"]= new[6]
    text["Sub_Ind7"]= new[7]
    # Dropping old Name columns 
    text.drop(columns =["Organization Industries"], inplace = True)

    #same thing but for investor column 
    new1 = text["Investor Names"].str.split(",", n = 11, expand = True) 
    # making separate first name column from new data frame 
    text["Investor1"]= new1[0] 
    # making separate last name column from new data frame 
    text["Investor1"]= new1[1] 
    text["Investor2"]= new1[2]
    text["Investor3"]= new1[3]
    text["Investor4"]= new1[4]
    text["Investor5"]= new1[5]
    text["Investor6"]= new1[6]
    text["Investor7"]= new1[7]
    text["Investor8"]= new1[8]
    text["Investor9"]= new1[9]
    text["Investor10"]= new1[10]
    text["Investor11"]= new1[11]
    # Dropping old Name columns 
    text.drop(columns =["Investor Names"], inplace = True)
    text.drop(columns =["Lead Investors"], inplace = True)
    text.drop(columns =["Number of Investors"], inplace = True)
    text = text[text['Money Raised'].notna()]
    text = text[text['Organization Location'].notna()]
    text = text[text['Primary Industry'].notna()]
    text = text[text['Money Raised Currency (in USD)'].notna()]
    



    return text 



In [ ]:
train = split_columns(train)
val = split_columns(val)
test = split_columns(test)


In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.isnull().sum()

In [ ]:
train['Raised Series A'].value_counts()

In [ ]:
train['Money Raised Currency (in USD)'].median()

In [ ]:
pd.set_option('max_colwidth', 150)
text = train[['Organization Description', 'Organization Name']]
text = text.sort_index()
text= pd.DataFrame(text)

text.head()

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:

# Let's take a look at the updated text
data_clean = pd.DataFrame(text['Organization Description'].apply(round1))
data_clean

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean['Organization Description'].apply(round2))
data_clean

In [ ]:
data_clean['Organization Name'] = train['Organization Name']
data_clean.set_index('Organization Name')

In [2]:
# use this link to start tomorrow: https://stackoverflow.com/questions/37807308/how-can-i-reduce-the-file-size-of-my-ipython-notebook
2+2 


4

In [ ]:
# Arrange data into X features matrix and y target vector
target = 'Raised Series A'
X_train = train.drop(columns=[target,'tokenized_Description','Announced Date'])
y_train = train[target]
X_val = val.drop(columns=[target,'tokenized_Description','Announced Date'])
y_val = val[target]
X_test = test.drop(columns=[target,'tokenized_Description','Announced Date'])
y_test = test[target]

In [ ]:
X_train.shape , y_train.shape

In [ ]:
val['Raised Series A'].value_counts()

In [ ]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=100,
                  random_state=42,
                  n_jobs=-1          
)
)

###scores = cross_val_score(pipeline, X_train, y_train, cv=10, 
                         ##scoring = 'average_precision')

###scores_val = cross_val_score(pipeline, X_val, y_val, cv=10, 
                         ##scoring = 'average_precision')'''


pipe = pipeline.fit(X_train, y_train);




In [ ]:
xgb_val = pipe.predict(X_val)
print("XGBoost Classifier val Recall Accuracy:", recall_score(y_val, xgb_val))
print("XGBoost Classifier val Accuracy Score:", accuracy_score(y_val, xgb_val))
print("XGBoost Classifier val Precision Accuracy:", precision_score(y_val, xgb_val))

In [ ]:
# when i ran predict(on X_val, it predicted that 10 of the 18 'Yes' values incorrectly. This could explain why the precision and recall accuracy is so low. 
unique_elements, counts_elements = np.unique(xgb_val, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
#make another 
target = 'Raised Series A'
X_train = train.drop(columns=[target,'tokenized_Description','Announced Date'])
y_train = train[target]
X_val = val.drop(columns=[target,'tokenized_Description','Announced Date'])
y_val = val[target]
X_test = test.drop(columns=[target,'tokenized_Description','Announced Date'])
y_test = test[target]

In [ ]:
#install imbalance package
pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

In [ ]:
#def corpora(d):
    #'Dictionary' Representation of all the words in our corpus
    #id2word = corpora.Dictionary(d["tokenized_Description"])
    #Removing extreme values from the dataset
    #id2word.filter_extremes(no_below=5, no_above=0.95)
    #bag of words(bow) representation of our corpus
    #corpus = [id2word.doc2bow(text) for text in d["tokenized_Description"]]
    #return d 

In [ ]:
#train = corpora(train)
#val = corpora(val)
#test = corpora(test)





In [ ]:
# precision, recall and F1
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
print('Recall', np.mean(recall), recall)
precision = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='precision')
print('Precision', np.mean(precision), precision)
f1 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1')
print('F1', np.mean(f1), f1)